In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from keras.utils import to_categorical
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv('/kaggle/input/liverpool-ion-switching/train.csv')
test=pd.read_csv('/kaggle/input/liverpool-ion-switching/test.csv')
sample=pd.read_csv('/kaggle/input/liverpool-ion-switching/sample_submission.csv')

In [ ]:
test.head(10)

In [ ]:
train.describe()

In [ ]:
train.notna().count()

In [ ]:
train.corr()

In [ ]:
train.nunique()

In [ ]:
fig,axes=plt.subplots(2,figsize=(15,15))
axes[0].plot(train['signal'])
axes[1].plot(train['open_channels'])


In [ ]:
fig,axes=plt.subplots(1,figsize=(15,10))
axes.plot(test['signal'])


In [ ]:
train['open_channels'].value_counts()

In [ ]:
train['signal'].value_counts()

In [ ]:
train.loc[train['signal'] == -2.7245][['signal','open_channels']]

In [ ]:
#train=train.drop_duplicates(['signal'])

In [ ]:
train.describe()

In [ ]:
train['open_channels'].value_counts().keys()

In [ ]:
my=pd.DataFrame({'signal':[],'open_channels':[]})
print(my)
for i in train['open_channels'].value_counts().keys():
    x=train.loc[train['open_channels']==i]['signal'].tolist()
    z=my['signal'].tolist()
    z.extend(x)
    my=pd.DataFrame({'signal':z,'open_channels':my['open_channels'].append(train.loc[train['open_channels']==i]['open_channels'])})

In [ ]:
my

In [ ]:
train['signal'].shape

In [ ]:
fig,ax=plt.subplots(2,figsize=(15,15))
ax[0].plot([i for i in range(train.shape[0])],train['signal'])
ax[1].plot([i for i in range(train.shape[0])],train['open_channels'])

In [ ]:
train_y=to_categorical(train['open_channels'])
train_x=train['signal'].to_numpy()

In [ ]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            x=sequence[i:len(sequence)].tolist()
         
            b=[0 for i in range(end_ix-len(sequence))]
            x.extend(b)
            seq_x=x
        else: 
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
# choose a number of time steps
n_steps = 10
# split into samples
train_x, y = split_sequence(train_x, n_steps)


In [ ]:
train_x=np.reshape(train_x,(train_x.shape[0],train_x.shape[1],1))
train_x.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, Dropout,LSTM,Dense
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model=Sequential()

    model.add(LSTM(256,activation='tanh',return_sequences=True))
    model.add(LSTM(128,activation='tanh'))
    model.add(Dense(11,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(train_x,train_y,shuffle=True,epochs=2,batch_size=5000)

    model.summary()


In [ ]:
#test=test.drop_duplicates(['signal'])
#test.shape

In [ ]:
test_x=test['signal'].to_numpy()
test_x, y = split_sequence(test_x, n_steps)
val=[]
test_x=test_x.reshape((test_x.shape[0],10,1))
val=model.predict_classes(test_x)


In [ ]:
print(val)

In [ ]:
plt.plot(test['signal'])

In [ ]:
plt.plot(val)

In [ ]:
sample

In [ ]:
dd=pd.DataFrame({'time':test['time'],'open_channels':val})

In [ ]:
dd

In [ ]:
dd.to_csv('submission.csv',index = False,float_format='%.4f')

In [ ]:
oupt=pd.read_csv('/kaggle/working/result.csv')


In [ ]:
oupt[:40]